<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [35]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [36]:
import folium
import pandas as pd

In [37]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [38]:
import os

print("Répertoire courant :", os.getcwd())
# lister les fichiers csv dans l'arborescence du notebook (peut prendre un peu de temps si gros)
for root, dirs, files in os.walk('.'):
    for f in files:
        if f.lower().endswith('.csv'):
            print(os.path.join(root, f))

Répertoire courant : /drive/DS0203EN/module_3
./dataset_part_3.csv


In [39]:
# Charger le dataset (après le fetch fait dans ton notebook)
spacex_df = pd.read_csv("dataset_part_3.csv")

# Vérifier les premières lignes
spacex_df.head()

,FlightNumber,PayloadMass,Flights,GridFins,Reused,Legs,Block,ReusedCount,Orbit_ES-L1,Orbit_GEO,...,Serial_B1048,Serial_B1049,Serial_B1050,Serial_B1051,Serial_B1054,Serial_B1056,Serial_B1058,Serial_B1059,Serial_B1060,Serial_B1062
0,1.0,6104.959412,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,525.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,677.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,500.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,3170.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Charger ton fichier encodé
df = pd.read_csv("dataset_part_3.csv")

print("Colonnes disponibles :")
print(df.columns.tolist())

# Identifier la colonne du site (peut-être one-hot encoded → ex: 'Launch_Site_CCAFS LC-40')
site_cols = [c for c in df.columns if 'site' in c.lower()]
print("\nColonnes liées aux sites :", site_cols)

# Créons un dictionnaire de coordonnées pour les principaux launch sites
site_coords = {
    'CCAFS LC-40': (28.562302, -80.577356),
    'KSC LC-39A': (28.608058, -80.604836),
    'VAFB SLC-4E': (34.632093, -120.610829),
    'CCAFS SLC-40': (28.563197, -80.576820),
    'Boca Chica': (25.9972, -97.1566)
}

# Construisons un DataFrame avec ces infos
launch_sites_df = pd.DataFrame([
    {"Launch Site": site, "Lat": lat, "Long": lon}
    for site, (lat, lon) in site_coords.items()
])

# Créer une carte centrée sur la moyenne des coordonnées
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=4)

# Ajouter des marqueurs pour chaque site
for _, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map
df

Colonnes disponibles :
['FlightNumber', 'PayloadMass', 'Flights', 'GridFins', 'Reused', 'Legs', 'Block', 'ReusedCount', 'Orbit_ES-L1', 'Orbit_GEO', 'Orbit_GTO', 'Orbit_HEO', 'Orbit_ISS', 'Orbit_LEO', 'Orbit_MEO', 'Orbit_PO', 'Orbit_SO', 'Orbit_SSO', 'Orbit_VLEO', 'LaunchSite_CCAFS SLC 40', 'LaunchSite_KSC LC 39A', 'LaunchSite_VAFB SLC 4E', 'LandingPad_5e9e3032383ecb267a34e7c7', 'LandingPad_5e9e3032383ecb554034e7c9', 'LandingPad_5e9e3032383ecb6bb234e7ca', 'LandingPad_5e9e3032383ecb761634e7cb', 'LandingPad_5e9e3033383ecbb9e534e7cc', 'Serial_B0003', 'Serial_B0005', 'Serial_B0007', 'Serial_B1003', 'Serial_B1004', 'Serial_B1005', 'Serial_B1006', 'Serial_B1007', 'Serial_B1008', 'Serial_B1010', 'Serial_B1011', 'Serial_B1012', 'Serial_B1013', 'Serial_B1015', 'Serial_B1016', 'Serial_B1017', 'Serial_B1018', 'Serial_B1019', 'Serial_B1020', 'Serial_B1021', 'Serial_B1022', 'Serial_B1023', 'Serial_B1025', 'Serial_B1026', 'Serial_B1028', 'Serial_B1029', 'Serial_B1030', 'Serial_B1031', 'Serial_B1032',

,FlightNumber,PayloadMass,Flights,GridFins,Reused,Legs,Block,ReusedCount,Orbit_ES-L1,Orbit_GEO,...,Serial_B1048,Serial_B1049,Serial_B1050,Serial_B1051,Serial_B1054,Serial_B1056,Serial_B1058,Serial_B1059,Serial_B1060,Serial_B1062
0,1.0,6104.959412,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,525.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,677.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,500.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,3170.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,86.0,15400.000000,2.0,1.0,1.0,1.0,5.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
86,87.0,15400.000000,3.0,1.0,1.0,1.0,5.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
87,88.0,15400.000000,6.0,1.0,1.0,1.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
88,89.0,15400.000000,3.0,1.0,1.0,1.0,5.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
## Task 1: Mark all launch sites on a map


import pandas as pd
import folium

# Charger ton dataset (si pas encore chargé)
df = pd.read_csv("dataset_part_3.csv")

# Dictionnaire : Launch Site -> (Lat, Long)
site_coords = {
    'CCAFS LC-40': (28.562302, -80.577356),
    'CCAFS SLC-40': (28.563197, -80.576820),
    'KSC LC-39A': (28.608058, -80.604836),
    'VAFB SLC-4E': (34.632093, -120.610829),
    'Boca Chica': (25.9972, -97.1566)
}

# Construire un DataFrame des launch sites
launch_sites_df = pd.DataFrame([
    {"Launch Site": site, "Lat": lat, "Long": lon}
    for site, (lat, lon) in site_coords.items()
])

# Créer une carte centrée sur la moyenne des coordonnées
site_map = folium.Map(
    location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()],
    zoom_start=4
)

# Ajouter des marqueurs pour chaque site
for _, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [42]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [43]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [44]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [45]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [46]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [47]:
# Task 2: Mark the success/failed launches for each site on the map
# Utiliser MarkerCluster pour grouper les points
marker_cluster = MarkerCluster().add_to(site_map)

# Ajouter un marqueur pour chaque lancement (succès = vert, échec = rouge)
for _, row in spacex_df.iterrows():
    outcome = "Success" if row['class'] == 1 else "Failure"
    color = "green" if row['class'] == 1 else "red"
    
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"{row['Launch Site']} ({outcome})"
    ).add_to(marker_cluster)

site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [48]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [49]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [50]:

# Créer une nouvelle colonne 'marker_color' selon la valeur de 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Vérification
spacex_df[['class', 'marker_color']].head(10)


,class,marker_color
0,0,red
1,0,red
2,0,red
3,0,red
4,0,red
5,0,red
6,0,red
7,0,red
8,0,red
9,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [51]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [52]:
import math

def haversine(coord1, coord2):
    # coordonnées en radians
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    R = 6371  # rayon moyen de la Terre en km
    return R * c

# Exemple : CCAFS LC-40 vers Cocoa Beach
launch_coord = (28.562302, -80.577356)
cocoa_beach = (28.3200, -80.6076)

distance_km = haversine(launch_coord, cocoa_beach)
print(f"Distance CCAFS LC-40 -> Cocoa Beach = {distance_km:.2f} km")


Distance CCAFS LC-40 -> Cocoa Beach = 27.10 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [53]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [54]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [55]:
import math

# Fonction Haversine pour calculer la distance en km
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon moyen de la Terre en km
    # conversion en radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Exemple : coordonnées de CCAFS LC-40
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# TODO : remplace ces coordonnées par celles relevées avec MousePosition
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calcul de la distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance du site {launch_site_lat},{launch_site_lon} à la côte ({coastline_lat},{coastline_lon}) = {distance_coastline:.2f} km")


Distance du site 28.562302,-80.577356 à la côte (28.56367,-80.57163) = 0.58 km


In [56]:
import folium
from folium.features import DivIcon

# Coordonnées du site (exemple : CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356
launch_coord = (launch_site_lat, launch_site_lon)

# Coordonnées du point de côte choisi avec MousePosition
coastline_lat = 28.56367
coastline_lon = -80.57163
coastline_coord = (coastline_lat, coastline_lon)

# Calcul de la distance avec la fonction Haversine
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Créer une carte centrée sur le site
site_map = folium.Map(location=launch_coord, zoom_start=14)

# Ajouter un marqueur pour le site de lancement
folium.Marker(
    location=launch_coord,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Ajouter un marqueur pour le point de côte
folium.Marker(
    location=coastline_coord,
    popup="Coastline Point",
    icon=folium.Icon(color='green')
).add_to(site_map)

# Ajouter une ligne entre le site et la côte
folium.PolyLine(
    [launch_coord, coastline_coord],
    color='blue',
    weight=2,
    dash_array='5,10'
).add_to(site_map)

# Ajouter un marker texte affichant la distance
distance_marker = folium.Marker(
    location=[(launch_site_lat+coastline_lat)/2, (launch_site_lon+coastline_lon)/2],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(site_map)

site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [57]:
# Coordonnées du site (exemple : CCAFS LC-40)
launch_coord = (28.562302, -80.577356)

# Coordonnées du point de côte choisi
coastline_coord = (28.56367, -80.57163)

# Créer une carte centrée sur le site
site_map = folium.Map(location=launch_coord, zoom_start=14)

# Ajouter les marqueurs
folium.Marker(launch_coord, popup="Launch Site", icon=folium.Icon(color="blue", icon="rocket")).add_to(site_map)
folium.Marker(coastline_coord, popup="Coastline", icon=folium.Icon(color="green")).add_to(site_map)

# Créer une ligne entre le site et la côte
coordinates = [launch_coord, coastline_coord]
lines = folium.PolyLine(locations=coordinates, weight=2, color="blue", dash_array="5,10")

# Ajouter la ligne à la carte
site_map.add_child(lines)

site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [58]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [59]:
import folium
from folium.features import DivIcon

# Fonction de distance (Haversine)
import math
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Coordonnées du site (ex: CCAFS LC-40)
launch_coord = (28.562302, -80.577356)

# Coordonnées de la ville proche (Cocoa Beach)
city_coord = (28.3200, -80.6076)

# Calcul de la distance
distance_city = calculate_distance(launch_coord[0], launch_coord[1], city_coord[0], city_coord[1])

# Créer une carte centrée sur le site
site_map = folium.Map(location=launch_coord, zoom_start=10)

# Marqueur pour le site de lancement
folium.Marker(
    location=launch_coord,
    popup="Launch Site",
    icon=folium.Icon(color="blue", icon="rocket")
).add_to(site_map)

# Marqueur pour la ville
folium.Marker(
    location=city_coord,
    popup="Cocoa Beach",
    icon=folium.Icon(color="purple", icon="info-sign")
).add_to(site_map)

# Ligne entre le site et la ville
folium.PolyLine(
    locations=[launch_coord, city_coord],
    weight=2,
    color="green",
    dash_array="5,10"
).add_to(site_map)

# Ajout d’un texte sur la ligne avec la distance
folium.Marker(
    location=[(launch_coord[0]+city_coord[0])/2, (launch_coord[1]+city_coord[1])/2],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_city),
    )
).add_to(site_map)

site_map


In [60]:
ratio = (df.groupby('Launch Site')['class']
           .agg(successes='sum', total='count'))
ratio['success_ratio'] = ratio['successes']/ratio['total']
best_site = ratio['success_ratio'].idxmax()
best_site, ratio.loc[best_site, 'success_ratio']

<class 'KeyError'>: 'Launch Site'

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
